In [91]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [92]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

imdb_id    budget   revenue             original_title  \
1046  tt1931533  15000000  19422261          Seven Psychopaths   
180   tt1205537  60000000  50549107  Jack Ryan: Shadow Recruit   
1068  tt1663143  14000000  11417362                   Fun Size   
957   tt0275022  12000000  61141030                 Crossroads   
362   tt1100119  40000000  28972508                     Shorts   

                                                   cast          director  \
1046  Colin Farrell|Sam Rockwell|Christopher Walken|...   Martin McDonagh   
180   Chris Pine|Keira Knightley|Kevin Costner|Nonso...   Kenneth Branagh   
1068  Victoria Justice|Jackson Nicoll|Chelsea Handle...     Josh Schwartz   
957   Britney Spears|Zoe Saldana|Taryn Manning|Anson...       Tamra Davis   
362   Jimmy Bennett|Kat Dennings|Jolie Vanier|Willia...  Robert Rodriguez   

                                                tagline  \
1046                      They Won't Take Any Shih Tzu.   
180                                       Trust no one.   
1068           Some people just can't handle Halloween.   
957                  Dreams change. Friends are forever   
362   Not So TALL Tales From The Director Of 'Spy Kids'   

                                               overview  runtime  \
1046  A struggling screenwriter inadvertently become...      110   
180   Jack Ryan, as a young covert CIA analyst, unco...      105   
1068  Wren (Victoria Justice) is invited to a Hallow...       87   
957   Three friends get together and bury a box maki...       93   
362   A young boy's discovery of a colorful, wish-gr...       89   

                                    genres  \
1046                 Comedy|Crime|Thriller   
180                  Action|Drama|Thriller   
1068                                Comedy   
957   Action|Adventure|Comedy|Drama|Family   
362                          Comedy|Family   

                                   production_companies release_date  \
1046  Blueprint Pictures|CBS Films|British Film Inst...     9/7/2012   
180   Paramount Pictures|Di Bonaventura Pictures|Mac...    1/15/2014   
1068  Paramount Pictures|Nickelodeon Movies|Anonymou...   10/25/2012   
957                        Paramount Pictures|MTV Films    2/15/2002   
362                               Warner Bros. Pictures    8/21/2009   

      vote_average  release_year  
1046           6.6          2012  
180            5.9          2014  
1068           5.4          2012  
957            5.1          2002  
362            4.8          2009

In [93]:
data.describe()

budget       revenue      runtime  vote_average  release_year
count  1.889000e+03  1.889000e+03  1889.000000   1889.000000   1889.000000
mean   5.431083e+07  1.553653e+08   109.658549      6.140762   2007.860773
std    4.858721e+07  2.146698e+08    18.017041      0.764763      4.468841
min    5.000000e+06  2.033165e+06    63.000000      3.300000   2000.000000
25%    2.000000e+07  3.456058e+07    97.000000      5.600000   2004.000000
50%    3.800000e+07  8.361541e+07   107.000000      6.100000   2008.000000
75%    7.200000e+07  1.782626e+08   120.000000      6.600000   2012.000000
max    3.800000e+08  2.781506e+09   214.000000      8.100000   2015.000000

# Предобработка

In [96]:
answers = {} # Cоздадим словарь для ответов


# Вопрос 6
# Добавление столбца profit (прибыль) в data
data['profit'] = data.revenue-data.budget 


# Вопрос 11 (Вариант 1)
# Получение вложенного списка жанров
split_genres = data.genres.apply(lambda x:str(x).split('|'))

# Вызов функции для объединения списков
from itertools import chain 
# Получение плоского списка жанров
list_genres_1 = list(chain.from_iterable(split_genres)) 

# Вопрос 11 (Варианты 2 и 3)
# Добавление столбца list_genres_2 (список жанров) в data
data['list_genres_2'] = data.genres.str.split('|')

# Преобразование каждого элемента списка жанров в строку 
# с сохранением индекса из data для каждой из строк
data_used_genres = data.explode('list_genres_2') 
                                               
# Вопрос 11 (Вариант 4)
# Создание отдельного датафрейма из разбитого по элементам строк
# столбца genres исходного data
genres_data = data['genres'].str.split('|',expand=True) 
# Переименование столбцов genres_data                                                        
genres_data.columns = ['genre_1','genre_2','genre_3','genre_4','genre_5'] 


# Вопрос 12
# Получение вложенного списка прибыльных жанров
split_genres_prof = data[data.profit>0].genres.apply(lambda x:str(x).split('|')) 
# Получение плоского списка прибыльных жанров                                                                                  
list_genres_prof_1 = list(chain.from_iterable(split_genres_prof))  


# Вопрос 13 (Вариант 1)
# Добавление столбца list_director (список режиссеров) в data
data['list_dir'] = data.director.str.split('|')

# Преобразование каждого элемента списка режиссеров в строку 
# с сохранением индекса из data для каждой из строк
data_used_dir = data.explode('list_dir')

# Группировка по столбцу list_dir датафрейма data
# с суммированием и сортировкой по убыванию кассовых сборов (revenue)
grouped_data_used_dir = data_used_dir.groupby(['list_dir'])['revenue'].\
                                              sum().sort_values(ascending=False)

# Вопрос 13 (Вариант 2)
# Группировка по столбцу director датафрейма data
# с суммированием и сортировкой по убыванию кассовых сборов (revenue)
grouped_dir_rev = data.groupby(['director'])['revenue'].\
                               sum().sort_values(ascending=False)

# Преобразование полученной серии в новый датафрейм dir_rev
dir_rev = grouped_dir_rev.reset_index()

# Получение вложенного списка режиссеров   
dir_rev.director = dir_rev.director.apply(lambda x:str(x).split('|'))

# Перегруппировка столбцов dir_rev
columnsTitles = ['revenue','director']
dir_rev = dir_rev.reindex(columns=columnsTitles)

# Создание нового датафрейма dir_rev_new с соответствующей суммой кассовых сборов 
# для каждого режиссера списка: 

dirs = [] # Создание список списков

# Создание строки для каждого режиссера в паре с соответствующей суммой кассовых сборов
dir_rev.apply(lambda dir: [dirs.append([dir['revenue'], nn]) 
                         for nn in dir.director], axis=1)

# Создание нового датафрейма из списка dirs, используя исходные имена столбцов
dir_rev_new = pd.DataFrame(dirs, columns=dir_rev.columns)

# Группировка по столбцу director датафрейма dir_rev_new
# с суммированием и сортировкой по убыванию кассовых сборов (revenue)
dir_rev_sum = dir_rev_new.groupby(['director'])['revenue'].\
                                  sum().sort_values(ascending=False)


# Вопрос 14
# Извлечение данных из data_used_dir по условию наличия жанра Action
data_used_dir_action = data_used_dir[data_used_dir.genres.str.contains("Action", na=False)]


# Вопрос 15
# Добавление столбца list_cast (список актеров) в data
data['list_cast'] = data.cast.str.split('|')

# Преобразование каждого элемента списка актеров в строку 
# с сохранением индекса из data для каждой из строк
data_used_cast = data.explode('list_cast')

# Извлечение данных из data_used_cast по условию релиза в 2012 году
data_used_cast_2012 = data_used_cast[data_used_cast.release_year==2012]

# Группировка по столбцу list_cast датафрейма data_used_cast_2012
# с суммированием и сортировкой по убыванию кассовых сборов (revenue)
grouped_data_used_cast_2012 = data_used_cast_2012.groupby(['list_cast'])['revenue'].\
                                                          sum().sort_values(ascending=False)


# Вопрос 16
# Извлечение данных из data_used_cast по условию бюджета выше среднего по данной выборке
data_used_cast_hb = data_used_cast[data_used_cast.budget>data_used_cast.budget.mean()]

# Группировка по столбцу list_cast датафрейма data_used_cast_hb
# с подсчетом и сортировкой по убыванию количества фильмов (imdb_id)
grouped_data_used_cast_hb = data_used_cast_hb.groupby(['list_cast'])['imdb_id'].\
                                                      count().sort_values(ascending=False)


# Вопрос 17
# Извлечение данных из data_used_genres по условию наличия Nicolas Cage
data_used_genres_Cage = data_used_genres[data_used_genres.\
                                         cast.str.contains('Nicolas Cage', na=False)]


# Вопрос 18
# Извлечение данных из data по условию наличия Paramount Pictures
data_Paramount = data[data.production_companies.str.contains("Paramount Pictures", na=False)]


# Вопрос 19
# Группировка по столбцу release_year датафрейма data
# с суммированием и сортировкой по убыванию кассовых сборов
year_rev_sum = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)


# Вопрос 20
# Извлечение данных из data по условию наличия Warner Bros
data_Warner = data[data.production_companies.str.contains("Warner Bros", na=False)]

# Группировка по столбцу release_year data_Warner
# с суммированием и сортировкой по убыванию прибыли
year_prof_sum_Warner = data_Warner.groupby(['release_year'])['profit'].\
                                            sum().sort_values(ascending=False)


# Вопрос 21 
# Преобразование формата даты столбца release_date исходного data в datetime формат
data['release_date']= pd.to_datetime(data['release_date'])

# Добавление столбца release_month в исходный датафрейм 
data['release_month'] = data['release_date'].dt.month

# Группировка по столбцу release_month data
# с подсчетом и сортировкой по убыванию суммарного количества фильмов
grouped_release_month = data.groupby(['release_month'])['imdb_id'].\
                                     count().sort_values(ascending=False)


# Вопрос 23
# Извлечение данных из data_used_dir по условию наличия зимних месяцев в столбце month
data_used_dir_winter = data_used_dir[(data_used_dir.release_month==12)|
                                     (data_used_dir.release_month==1)|(data_used_dir.release_month==2)]


# Вопрос 24
# Добавление столбца len_title (длина названия) в data
data['len_title'] = data.original_title.str.len()

# Добавление столбца list_companies (список кинокомпаний) в data
data['list_companies'] = data.production_companies.str.split('|')

# Преобразование каждого элемента списка кинокомпаний в строку 
# с сохранением индекса из data для каждой из строк
data_used_companies = data.explode('list_companies')

# Группировка по столбцу list_companies датафрейма data_used_companies
# с подсчетом и сортировкой по убыванию максимальной длины названий фильмов (len_title)
grouped_data_used_comp_len = data_used_companies.groupby(['list_companies'])['len_title'].\
                                                      max().sort_values(ascending=False)

# Вопрос 25
# Добавление столбца overview_words (количество слов в описании) в data
data['overview_words'] = data.overview.str.split().apply(lambda x:len(x))

# Группировка по столбцу list_companies датафрейма data_used_companies
# с подсчетом и сортировкой по убыванию средней длины описания по количеству слов (overview_words)
grouped_data_used_comp_words = data_used_companies.groupby(['list_companies'])['overview_words'].\
                                                      mean().sort_values(ascending=False)


# Вопрос 26
# Извлечение данных из data в части столбцов original_title и vote_average 
# с сортировкой по убыванию рейтинга
title_vote = data[['original_title','vote_average']].sort_values(['vote_average'],ascending=False)


# Вопрос 27
# Получение вложенного списка актеров
cast = data.cast.str.split('|').tolist()

# Вызов функции для создания комбинации длиной r из iterable без повторяющихся элементов
from itertools import combinations
# Получение вложенного списка комбинаций пар актеров
cast_couples = []
for i in range(len(cast)):
    cast_couples.append(list(combinations(cast[i],2)))

# Получение плоского списка комбинаций пар актеров
list_cast_couples = list(chain.from_iterable(cast_couples))  

# 1. У какого фильма из списка самый большой бюджет?

In [97]:
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # +

ВАРИАНТ 1

In [98]:
data[data.budget==data.budget.max()]

imdb_id     budget     revenue  \
723  tt1298650  380000000  1021683000   

                                  original_title  \
723  Pirates of the Caribbean: On Stranger Tides   

                                                  cast      director  \
723  Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...  Rob Marshall   

                         tagline  \
723  Live Forever Or Die Trying.   

                                              overview  runtime  \
723  Captain Jack Sparrow crosses paths with a woma...      136   

                       genres  ... vote_average release_year     profit  \
723  Adventure|Action|Fantasy  ...          6.3         2011  641683000   

                    list_genres_2        list_dir  \
723  [Adventure, Action, Fantasy]  [Rob Marshall]   

                                             list_cast release_month  \
723  [Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...             5   

    len_title                                     list_companies  \
723        43  [Walt Disney Pictures, Jerry Bruckheimer Films...   

     overview_words  
723              75  

[1 rows x 22 columns]

ВАРИАНТ 2

In [99]:
data.query('budget==budget.max()')

imdb_id     budget     revenue  \
723  tt1298650  380000000  1021683000   

                                  original_title  \
723  Pirates of the Caribbean: On Stranger Tides   

                                                  cast      director  \
723  Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...  Rob Marshall   

                         tagline  \
723  Live Forever Or Die Trying.   

                                              overview  runtime  \
723  Captain Jack Sparrow crosses paths with a woma...      136   

                       genres  ... vote_average release_year     profit  \
723  Adventure|Action|Fantasy  ...          6.3         2011  641683000   

                    list_genres_2        list_dir  \
723  [Adventure, Action, Fantasy]  [Rob Marshall]   

                                             list_cast release_month  \
723  [Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...             5   

    len_title                                     list_companies  \
723        43  [Walt Disney Pictures, Jerry Bruckheimer Films...   

     overview_words  
723              75  

[1 rows x 22 columns]

# 2. Какой из фильмов самый длительный (в минутах)?

In [100]:
answers['2'] = '1157. Gods and Generals (tt0279111)' # +

In [101]:
data[data.runtime==data.runtime.max()]

imdb_id    budget   revenue     original_title  \
1157  tt0279111  56000000  12923936  Gods and Generals   

                                                   cast           director  \
1157  Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...  Ronald F. Maxwell   

                                  tagline  \
1157  The nations heart was touched by...   

                                               overview  runtime  \
1157  The film centers mostly around the personal an...      214   

                 genres  ... vote_average release_year    profit  \
1157  Drama|History|War  ...          5.8         2003 -43076064   

              list_genres_2             list_dir  \
1157  [Drama, History, War]  [Ronald F. Maxwell]   

                                              list_cast release_month  \
1157  [Stephen Lang, Jeff Daniels, Robert Duvall, Ke...             2   

     len_title                         list_companies  overview_words  
1157        17  [Turner Pictures, Antietam Filmworks]              48  

[1 rows x 22 columns]

# 3. Какой из фильмов самый короткий (в минутах)?





In [102]:
answers['3'] = '768. Winnie the Pooh (tt1449283)' # +

In [103]:
data.query('runtime==runtime.min()')

imdb_id    budget   revenue   original_title  \
768  tt1449283  30000000  14460000  Winnie the Pooh   

                                                  cast  \
768  Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...   

                      director   tagline  \
768  Stephen Anderson|Don Hall  Oh Pooh.   

                                              overview  runtime  \
768  During an ordinary day in Hundred Acre Wood, W...       63   

               genres  ... vote_average release_year    profit  \
768  Animation|Family  ...          6.8         2011 -15540000   

           list_genres_2                      list_dir  \
768  [Animation, Family]  [Stephen Anderson, Don Hall]   

                                             list_cast release_month  \
768  [Jim Cummings, Travis Oates, Jim Cummings, Bud...             4   

    len_title                                     list_companies  \
768        15  [Walt Disney Pictures, Walt Disney Animation S...   

     overview_words  
768              52  

[1 rows x 22 columns]

# 4. Какова средняя длительность фильмов?


In [104]:
answers['4'] = '110' # +

ВАРИАНТ 1

In [105]:
data.runtime.mean()

109.6585494970884

ВАРИАНТ 2

In [106]:
data.describe()

budget       revenue      runtime  vote_average  release_year  \
count  1.889000e+03  1.889000e+03  1889.000000   1889.000000   1889.000000   
mean   5.431083e+07  1.553653e+08   109.658549      6.140762   2007.860773   
std    4.858721e+07  2.146698e+08    18.017041      0.764763      4.468841   
min    5.000000e+06  2.033165e+06    63.000000      3.300000   2000.000000   
25%    2.000000e+07  3.456058e+07    97.000000      5.600000   2004.000000   
50%    3.800000e+07  8.361541e+07   107.000000      6.100000   2008.000000   
75%    7.200000e+07  1.782626e+08   120.000000      6.600000   2012.000000   
max    3.800000e+08  2.781506e+09   214.000000      8.100000   2015.000000   

             profit  release_month    len_title  overview_words  
count  1.889000e+03    1889.000000  1889.000000     1889.000000  
mean   1.010545e+08       6.933298    15.037057       52.381154  
std    1.830733e+08       3.390196     8.552660       26.589348  
min   -1.657101e+08       1.000000     1.000000       10.000000  
25%    5.000000e+06       4.000000     9.000000       32.000000  
50%    4.400041e+07       7.000000    13.000000       48.000000  
75%    1.201842e+08      10.000000    18.000000       67.000000  
max    2.544506e+09      12.000000    83.000000      175.000000

# 5. Каково медианное значение длительности фильмов? 

In [107]:
answers['5'] = '107' # +

ВАРИАНТ 1

In [108]:
data.runtime.median()

107.0

ВАРИАНТ 2

In [109]:
data.describe()

budget       revenue      runtime  vote_average  release_year  \
count  1.889000e+03  1.889000e+03  1889.000000   1889.000000   1889.000000   
mean   5.431083e+07  1.553653e+08   109.658549      6.140762   2007.860773   
std    4.858721e+07  2.146698e+08    18.017041      0.764763      4.468841   
min    5.000000e+06  2.033165e+06    63.000000      3.300000   2000.000000   
25%    2.000000e+07  3.456058e+07    97.000000      5.600000   2004.000000   
50%    3.800000e+07  8.361541e+07   107.000000      6.100000   2008.000000   
75%    7.200000e+07  1.782626e+08   120.000000      6.600000   2012.000000   
max    3.800000e+08  2.781506e+09   214.000000      8.100000   2015.000000   

             profit  release_month    len_title  overview_words  
count  1.889000e+03    1889.000000  1889.000000     1889.000000  
mean   1.010545e+08       6.933298    15.037057       52.381154  
std    1.830733e+08       3.390196     8.552660       26.589348  
min   -1.657101e+08       1.000000     1.000000       10.000000  
25%    5.000000e+06       4.000000     9.000000       32.000000  
50%    4.400041e+07       7.000000    13.000000       48.000000  
75%    1.201842e+08      10.000000    18.000000       67.000000  
max    2.544506e+09      12.000000    83.000000      175.000000

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [110]:
answers['6'] = '239. Avatar (tt0499549)' # +

ВАРИАНТ 1

In [111]:
data[data.profit==data.profit.max()]

imdb_id     budget     revenue original_title  \
239  tt0499549  237000000  2781505847         Avatar   

                                                  cast       director  \
239  Sam Worthington|Zoe Saldana|Sigourney Weaver|S...  James Cameron   

                         tagline  \
239  Enter the World of Pandora.   

                                              overview  runtime  \
239  In the 22nd century, a paraplegic Marine is di...      162   

                                       genres  ... vote_average release_year  \
239  Action|Adventure|Fantasy|Science Fiction  ...          7.1         2009   

         profit                                  list_genres_2  \
239  2544505847  [Action, Adventure, Fantasy, Science Fiction]   

            list_dir                                          list_cast  \
239  [James Cameron]  [Sam Worthington, Zoe Saldana, Sigourney Weave...   

    release_month len_title  \
239            12         6   

                                        list_companies  overview_words  
239  [Ingenious Film Partners, Twentieth Century Fo...              28  

[1 rows x 22 columns]

ВАРИАНТ 2

In [112]:
data.query('profit==profit.max()')

imdb_id     budget     revenue original_title  \
239  tt0499549  237000000  2781505847         Avatar   

                                                  cast       director  \
239  Sam Worthington|Zoe Saldana|Sigourney Weaver|S...  James Cameron   

                         tagline  \
239  Enter the World of Pandora.   

                                              overview  runtime  \
239  In the 22nd century, a paraplegic Marine is di...      162   

                                       genres  ... vote_average release_year  \
239  Action|Adventure|Fantasy|Science Fiction  ...          7.1         2009   

         profit                                  list_genres_2  \
239  2544505847  [Action, Adventure, Fantasy, Science Fiction]   

            list_dir                                          list_cast  \
239  [James Cameron]  [Sam Worthington, Zoe Saldana, Sigourney Weave...   

    release_month len_title  \
239            12         6   

                                        list_companies  overview_words  
239  [Ingenious Film Partners, Twentieth Century Fo...              28  

[1 rows x 22 columns]

# 7. Какой фильм самый убыточный? 

In [113]:
answers['7'] = '1245. The Lone Ranger (tt1210819)' # +

In [114]:
data[data.profit==data.profit.min()]

imdb_id     budget   revenue   original_title  \
1245  tt1210819  255000000  89289910  The Lone Ranger   

                                                   cast        director  \
1245  Johnny Depp|Armie Hammer|William Fichtner|Hele...  Gore Verbinski   

                      tagline  \
1245  Never Take Off the Mask   

                                               overview  runtime  \
1245  The Texas Rangers chase down a gang of outlaws...      149   

                        genres  ... vote_average release_year     profit  \
1245  Action|Adventure|Western  ...          6.0         2013 -165710090   

                     list_genres_2          list_dir  \
1245  [Action, Adventure, Western]  [Gore Verbinski]   

                                              list_cast release_month  \
1245  [Johnny Depp, Armie Hammer, William Fichtner, ...             7   

     len_title                                     list_companies  \
1245        15  [Walt Disney Pictures, Jerry Bruckheimer Films...   

      overview_words  
1245              68  

[1 rows x 22 columns]

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [115]:
answers['8'] = '1478' # +

ВАРИАНТ 1

In [116]:
data[data.profit>0].imdb_id.count()

1478

ВАРИАНТ 2

In [117]:
data.query('profit>0').imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [118]:
answers['9'] = '599. The Dark Knight (tt0468569)' # +

In [119]:
data_release = data[data.release_year==2008]
data_release[data_release.revenue==data_release.revenue.max()]

imdb_id     budget     revenue   original_title  \
599  tt0468569  185000000  1001921825  The Dark Knight   

                                                  cast           director  \
599  Christian Bale|Michael Caine|Heath Ledger|Aaro...  Christopher Nolan   

             tagline                                           overview  \
599  Why So Serious?  Batman raises the stakes in his war on crime. ...   

     runtime                       genres  ... vote_average release_year  \
599      152  Drama|Action|Crime|Thriller  ...          8.1         2008   

        profit                     list_genres_2             list_dir  \
599  816921825  [Drama, Action, Crime, Thriller]  [Christopher Nolan]   

                                             list_cast release_month  \
599  [Christian Bale, Michael Caine, Heath Ledger, ...             7   

    len_title                                     list_companies  \
599        15  [DC Comics, Legendary Pictures, Warner Bros., ...   

     overview_words  
599              67  

[1 rows x 22 columns]

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [120]:
answers['10'] = '1215. The Lone Ranger (tt1210819)' # +

In [121]:
data_release = data.query('2012<=release_year<=2014')
data_release.query('profit==profit.min()')

imdb_id     budget   revenue   original_title  \
1245  tt1210819  255000000  89289910  The Lone Ranger   

                                                   cast        director  \
1245  Johnny Depp|Armie Hammer|William Fichtner|Hele...  Gore Verbinski   

                      tagline  \
1245  Never Take Off the Mask   

                                               overview  runtime  \
1245  The Texas Rangers chase down a gang of outlaws...      149   

                        genres  ... vote_average release_year     profit  \
1245  Action|Adventure|Western  ...          6.0         2013 -165710090   

                     list_genres_2          list_dir  \
1245  [Action, Adventure, Western]  [Gore Verbinski]   

                                              list_cast release_month  \
1245  [Johnny Depp, Armie Hammer, William Fichtner, ...             7   

     len_title                                     list_companies  \
1245        15  [Walt Disney Pictures, Jerry Bruckheimer Films...   

      overview_words  
1245              68  

[1 rows x 22 columns]

# 11. Какого жанра фильмов больше всего?

In [122]:
answers['11'] = 'Drama' # +

ВАРИАНТ 1

In [123]:
Counter(list_genres_1).most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

ВАРИАНТ 2

In [124]:
Counter(data_used_genres.list_genres_2).most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

ВАРИАНТ 3

In [125]:
data_used_genres.list_genres_2.value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: list_genres_2, dtype: int64

ВАРИАНТ 4

In [126]:
genres_sum = (Counter(genres_data.genre_1)+Counter(genres_data.genre_2) 
           +Counter(genres_data.genre_3)+Counter(genres_data.genre_4)+Counter(genres_data.genre_5))
Counter(genres_sum).most_common()

[(None, 4338),
 ('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [127]:
answers['12'] = 'Drama' # + 

In [128]:
Counter(list_genres_prof_1).most_common()

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [129]:
answers['13'] = 'Peter Jackson' # +

ВАРИАНТ 1

In [130]:
grouped_data_used_dir[grouped_data_used_dir==max(grouped_data_used_dir)]

list_dir
Peter Jackson    6490593685
Name: revenue, dtype: int64

ВАРИАНТ 2

In [131]:
dir_rev_sum[dir_rev_sum==max(dir_rev_sum)]

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [132]:
answers['14'] = 'Robert Rodriguez' # +

ВАРИАНТ 1

In [133]:
Counter(data_used_dir_action.list_dir).most_common()

[('Robert Rodriguez', 9),
 ('Michael Bay', 7),
 ('Paul W.S. Anderson', 7),
 ('Antoine Fuqua', 6),
 ('Ridley Scott', 6),
 ('Brett Ratner', 5),
 ('Quentin Tarantino', 5),
 ('Zack Snyder', 5),
 ('Roland Emmerich', 5),
 ('Andrzej Bartkowiak', 5),
 ('Louis Leterrier', 5),
 ('Tony Scott', 5),
 ('Paul Greengrass', 5),
 ('Peter Jackson', 5),
 ('Lee Tamahori', 5),
 ('Gore Verbinski', 5),
 ('Rob Cohen', 5),
 ('Peter Berg', 5),
 ('J.J. Abrams', 4),
 ('Lana Wachowski', 4),
 ('Lilly Wachowski', 4),
 ('Olivier Megaton', 4),
 ('Guy Ritchie', 4),
 ('Bryan Singer', 4),
 ('Tim Story', 4),
 ('Renny Harlin', 4),
 ('Mark Neveldine', 4),
 ('Brian Taylor', 4),
 ('Dominic Sena', 4),
 ('Justin Lin', 4),
 ('Christopher Nolan', 4),
 ('Simon West', 4),
 ('John Moore', 4),
 ('Steven Spielberg', 4),
 ('Martin Campbell', 4),
 ('John Singleton', 4),
 ('Len Wiseman', 4),
 ('Brad Bird', 3),
 ('Matthew Vaughn', 3),
 ('Neill Blomkamp', 3),
 ('Jaume Collet-Serra', 3),
 ('Breck Eisner', 3),
 ('Pierre Morel', 3),
 ('David A

ВАРИАНТ 2

In [134]:
data_used_dir_action.list_dir.value_counts()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Joe Russo             1
Chuck Russell         1
BjÃ¶rn Stein          1
Lexi Alexander        1
Dan Bradley           1
Name: list_dir, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [135]:
answers['15'] = 'Chris Hemsworth' # +

In [136]:
grouped_data_used_cast_2012[grouped_data_used_cast_2012==max(grouped_data_used_cast_2012)]

list_cast
Chris Hemsworth    2027450773
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [137]:
answers['16'] = 'Matt Damon' # +

In [138]:
grouped_data_used_cast_hb[grouped_data_used_cast_hb==max(grouped_data_used_cast_hb)]

list_cast
Matt Damon    18
Name: imdb_id, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [139]:
answers['17'] = 'Action' # +

In [140]:
Counter(data_used_genres_Cage.list_genres_2).most_common()

[('Action', 17),
 ('Thriller', 15),
 ('Drama', 12),
 ('Crime', 10),
 ('Fantasy', 8),
 ('Adventure', 7),
 ('Comedy', 6),
 ('Science Fiction', 4),
 ('Mystery', 3),
 ('Animation', 3),
 ('Family', 3),
 ('History', 2),
 ('War', 1),
 ('Horror', 1),
 ('Romance', 1)]

# 18. Самый убыточный фильм от Paramount Pictures

In [141]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)' # +

In [142]:
data_Paramount[data_Paramount.profit==data_Paramount.profit.min()]

imdb_id     budget   revenue        original_title  \
925  tt0267626  100000000  35168966  K-19: The Widowmaker   

                                                  cast         director  \
925  Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...  Kathryn Bigelow   

                      tagline  \
925  Fate has found its hero.   

                                              overview  runtime  \
925  When Russia's first nuclear submarine malfunct...      138   

                     genres  ... vote_average release_year    profit  \
925  Thriller|Drama|History  ...          6.0         2002 -64831034   

                  list_genres_2           list_dir  \
925  [Thriller, Drama, History]  [Kathryn Bigelow]   

                                             list_cast release_month  \
925  [Harrison Ford, Liam Neeson, Peter Sarsgaard, ...             7   

    len_title                                     list_companies  \
925        20  [Paramount Pictures, Intermedia Films, Nationa...   

     overview_words  
925              23  

[1 rows x 22 columns]

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [143]:
answers['19'] = '2015' # +

In [144]:
year_rev_sum[year_rev_sum==year_rev_sum.max()]

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [145]:
answers['20'] = '2014' # +

In [146]:
year_prof_sum_Warner[year_prof_sum_Warner==year_prof_sum_Warner.max()]

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [147]:
answers['21'] = 'Сентябрь' # +

In [148]:
grouped_release_month[grouped_release_month==max(grouped_release_month)]

release_month
9    227
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [149]:
answers['22'] = '450' # +

In [150]:
grouped_release_month[6]+grouped_release_month[7]+grouped_release_month[8]

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [151]:
answers['23'] = 'Peter Jackson' # +

In [152]:
Counter(data_used_dir_winter.list_dir).most_common()

[('Peter Jackson', 7),
 ('Clint Eastwood', 6),
 ('Steven Soderbergh', 6),
 ('Shawn Levy', 4),
 ('Nancy Meyers', 4),
 ('Martin Scorsese', 4),
 ('Adam Shankman', 4),
 ('Ridley Scott', 3),
 ('Rob Marshall', 3),
 ('Tim Burton', 3),
 ('Steve Carr', 3),
 ('Patrick Lussier', 3),
 ('Donald Petrie', 3),
 ('Marc Lawrence', 3),
 ('Lasse HallstrÃ¶m', 3),
 ('Edward Zwick', 3),
 ('Jason Friedberg', 3),
 ('Aaron Seltzer', 3),
 ('Peter Farrelly', 3),
 ('Steven Spielberg', 3),
 ('Wayne Wang', 3),
 ('Quentin Tarantino', 2),
 ('Adam McKay', 2),
 ('Michael Mann', 2),
 ('Glenn Ficarra', 2),
 ('John Requa', 2),
 ('Pierre Morel', 2),
 ('Wes Anderson', 2),
 ('Paul Thomas Anderson', 2),
 ('Jaume Collet-Serra', 2),
 ('George Clooney', 2),
 ('Shana Feste', 2),
 ('Ken Kwapis', 2),
 ('Henry Selick', 2),
 ('P.J. Hogan', 2),
 ('David S. Goyer', 2),
 ('Paul McGuigan', 2),
 ('Marcus Nispel', 2),
 ('Andrzej Bartkowiak', 2),
 ('Scott Cooper', 2),
 ('Allen Hughes', 2),
 ('Garry Marshall', 2),
 ('David O. Russell', 2),
 (

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [153]:
answers['24'] = 'Four By Two Productions' # +

In [154]:
grouped_data_used_comp_len

list_companies
Four By Two Productions                   83
Twentieth Century Fox Film Corporation    83
Walden Media                              62
Walt Disney                               62
Jim Henson Company, The                   59
                                          ..
Everest Entertainment                      3
Berlanti Productions                       3
XM2 Productions                            2
Global Entertainment Group                 2
Ixtlan Productions                         2
Name: len_title, Length: 1771, dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [155]:
answers['25'] = 'Midnight Picture Show' # +

In [156]:
grouped_data_used_comp_words

list_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
98 MPH Productions                       159.0
Heineken Branded Entertainment           159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: overview_words, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [157]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave' # +

In [158]:
title_vote.head(18) # первые 18 строк составляют 1 % от 1889 строк датафрейма

original_title  vote_average
599                                 The Dark Knight           8.1
118                                    Interstellar           8.0
125                              The Imitation Game           8.0
9                                        Inside Out           8.0
34                                             Room           8.0
1183                        The Wolf of Wall Street           7.9
128                                       Gone Girl           7.9
1191                               12 Years a Slave           7.9
119                         Guardians of the Galaxy           7.9
1081  The Lord of the Rings: The Return of the King           7.9
1800                                        Memento           7.9
370                                       Inception           7.9
872                                     The Pianist           7.9
138                        The Grand Budapest Hotel           7.9
1188                                            Her           7.8
28                                        Spotlight           7.8
124                                      Big Hero 6           7.8
177                          The Fault in Our Stars           7.8

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [165]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint' # +

In [166]:
Counter(list_cast_couples).most_common()

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5),
 (('Robert Pattinson', 'Taylor Lautner'), 5),
 (('Jennifer Lawrence', 'Josh Hutcherson'), 4),
 (('Jennifer Lawrence', 'Liam Hemsworth'), 4),
 (('Jennifer Lawrence', 'Woody Harrelson'), 4),
 (('Josh Hutcherson', 'Liam Hemsworth'), 4),
 (('Josh Hutcherson', 'Woody Harrelson'), 4),
 (('Liam Hemsworth', 'Woody Harrelson'), 4),
 (('Tobin Bell', 'Costas Mandylor'), 4),
 (('Tobin Bell', 'Betsy Russell'), 4),
 (('Costas Mandylor', 'Betsy Russell'), 4),
 (('Vin Diesel', 'Jordana Brewster'), 4),
 (('Paul Walker', 'Jordana Brewster'), 4),
 (('Mike Myers', 'Eddie Murphy'), 4),
 (('Mike Myers', 'Cameron Diaz'), 4),
 (('Eddi

# Submission

In [167]:
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1215. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [168]:
len(answers)

27